In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sb
from matplotlib import pyplot as plt
import catboost
from catboost import *

In [ ]:
train = pd.read_csv('../input/amazon-employee-access-challenge/train.csv')
test = pd.read_csv('../input/amazon-employee-access-challenge/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
y = train['ACTION']
X = train.drop(['ACTION','RESOURCE',"ROLE_CODE"], axis=1, inplace=False)

In [ ]:
for cols in X.columns:
    plt.scatter(X[cols].tolist(),y.tolist())
    plt.xlabel(cols)
    plt.show()

In [ ]:
plot_database = pd.concat([X,y], axis=1)
sb.pairplot(plot_database)

In [ ]:
test['ROLE_CODE'].unique()

In [ ]:
df = train.drop(columns=['ACTION'])
train_x = train.drop(columns=['ACTION'])
train_y = train['ACTION'] 
test_x = test.drop(columns=['id'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(train_x, train_y,random_state=0)

In [ ]:
# X_train.shape

In [ ]:
# y_train.shape

In [ ]:
X_test.shape

In [ ]:
# y_test.shape

In [ ]:
# from sklearn import linear_model as lm
# from sklearn.metrics import accuracy_score
# model1 = lm.LogisticRegression()
# model1.fit(X_train, y_train)
# predictions = model1.predict(X_test)
# model1.score(X_test, y_test)
# accuracy_score(predictions, y_test)

In [ ]:
# from sklearn import tree
# from sklearn.tree import DecisionTreeClassifier
# model2 = DecisionTreeClassifier(max_depth=30)
# clf2= model2.fit(X_train, y_train)
# predictions = clf2.predict(test_x)
# clf2.score(X_test,y_test)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# model3 = RandomForestClassifier(n_estimators = 300)
# clf3 = model3.fit(X_train, y_train)
# predictions = clf3.predict_proba(test_x)
# clf3.score(X_test, y_test)

In [ ]:
os.chdir('/kaggle/working')
os.makedirs('/kaggle/working/amazon')
os.listdir('.')

In [ ]:
from catboost.utils import create_cd
feature_names = dict()
for column, name in enumerate(train):
    if column == 0:
        continue
    feature_names[column - 1] = name
dataset_dir = './amazon'   
create_cd(label=0, cat_features=list(range(1, train.columns.shape[0])),feature_names=feature_names,output_path=os.path.join(dataset_dir,'train.cd'))

In [ ]:
X = train.drop(columns=['ACTION'])
y = train.ACTION
cat_features = list(range(0, X.shape[1]))
print(cat_features)

In [ ]:
pool1 = Pool(data=X, label=y, cat_features=cat_features)
pool2 = Pool(
    data=os.path.join('/kaggle/input/amazon-employee-access-challenge/', 'train.csv'), 
    delimiter=',', 
    column_description=os.path.join(dataset_dir, 'train.cd'),
    has_header=True
)
pool3 = Pool(data=X, cat_features=cat_features)
X_prepared = X.values.astype(str).astype(object)
pool4 = Pool(
    data=FeaturesData(
        cat_feature_data=X_prepared,
        cat_feature_names=list(X)
    ),
    label=y.values
)

In [ ]:
print('Dataset shape')
print('dataset 1:' + str(pool1.shape) +
      '\ndataset 2:' + str(pool2.shape) + 
      '\ndataset 3:' + str(pool3.shape) +
      '\ndataset 4: ' + str(pool4.shape))

print('\n')
print('Column names')
print('dataset 1:')
print(pool1.get_feature_names()) 
print('\ndataset 2:')
print(pool2.get_feature_names())
print('\ndataset 3:')
print(pool3.get_feature_names())
print('\ndataset 4:')
print(pool4.get_feature_names())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7, random_state=1234)

In [ ]:
from catboost import CatBoostClassifier
model4= CatBoostClassifier(iterations=3000,random_seed=1,learning_rate=0.03,loss_function = 'CrossEntropy')
model4.fit(X_train, y_train,cat_features=cat_features,eval_set=(X_validation, y_validation),verbose=50)

In [ ]:
from catboost import cv

params = {}
params['loss_function'] = 'Logloss'
params['iterations'] = 93
params['custom_loss'] = 'AUC'
params['random_seed'] = 63
params['learning_rate'] = 0.5

cv_data = cv(
    params = params,
    pool = Pool(X, label=y, cat_features=cat_features),
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    stratified=False,
    verbose=False,plot=True
)

In [ ]:
cv_data

In [ ]:
best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print('Best validation Logloss score, not stratified: {:.4f}±{:.4f} on step {}'.format(
    best_value,
    cv_data['test-Logloss-std'][best_iter],
    best_iter)
)

In [ ]:
test = pd.read_csv('../input/amazon-employee-access-challenge/test.csv')
preds= model4.predict_proba(test_x)

In [ ]:
np.unique(preds, return_counts= True)

In [ ]:
preds = preds[:,1]

In [ ]:
os.chdir('/kaggle/working/')
os.curdir
submission = pd.DataFrame(preds)


submission = submission.rename(columns={0:'Action'})
submission.index = range(1, 58922,1)
submission = submission.rename_axis('Id')
submission.to_csv('submission.csv')